In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import os
path = "drive/MyDrive/team8/task2/"
os.listdir(path)

['test_data.csv',
 'val_label.csv',
 'train_data.csv',
 'val_data.csv',
 'test_label.csv',
 'train_label.csv']

In [ ]:
X_train=pd.read_csv(path+'train_data.csv')
y_train=pd.read_csv(path+'train_label.csv')
X_test=pd.read_csv(path+'test_data.csv')
y_test=pd.read_csv(path+'test_label.csv')
X_valid=pd.read_csv(path+'val_data.csv')
y_valid=pd.read_csv(path+'val_label.csv')

In [ ]:
X_train

,7.446771306225266618e-02,1.096294050519303898e-01,1.569654709788903535e-01,2.310947500725501480e-01,1.126853110649048556e-01,3.853791063923046878e-02,2.145509164448148806e-02,5.811229003137997895e-02,1.782218494570268208e-01,1.864831935583493028e-01,...,2.124081347816168253e-01,8.751326953217486437e-02,1.514482642673911361e-01,2.310947500725501480e-01.5,2.310947500725501480e-01.6,2.310947500725501480e-01.7,2.310947500725501480e-01.8,2.310947500725501480e-01.9,2.298241312480487075e-01,1.442986070531580012e-01
0,0.160090,0.236524,0.241060,0.212123,0.143671,0.084047,0.053078,0.096586,0.183855,0.227792,...,0.227918,0.067308,0.084922,0.083279,0.136707,0.060614,0.186163,0.241060,0.111923,0.148401
1,0.232613,0.156637,0.072718,0.196045,0.143884,0.121276,0.059695,0.056117,0.036234,0.232613,...,0.112994,0.149468,0.232613,0.232613,0.197228,0.173609,0.200801,0.184820,0.119906,0.154592
2,0.270416,0.270416,0.270416,0.270416,0.110150,0.100266,0.172866,0.101392,0.176556,0.041513,...,0.112757,0.017514,0.038029,0.046072,0.046432,0.122355,0.270416,0.270416,0.270416,0.085594
3,0.256831,0.010630,0.063891,0.102741,0.048164,0.011751,0.006206,0.050408,0.153415,0.256831,...,0.138603,0.256831,0.256831,0.175417,0.126874,0.153245,0.036332,0.089795,0.183911,0.256831
4,0.251023,0.142318,0.121610,0.062252,0.089635,0.081542,0.053552,0.094480,0.251023,0.251023,...,0.166349,0.225742,0.251023,0.087291,0.030133,0.144603,0.075943,0.185409,0.251023,0.251023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1994,0.171226,0.219380,0.216864,0.190345,0.126700,0.109725,0.123107,0.146005,0.065662,0.137462,...,0.250035,0.102743,0.103784,0.174426,0.132063,0.240602,0.125937,0.196811,0.123043,0.250035
1995,0.146548,0.125712,0.226871,0.111810,0.339496,0.041248,0.025843,0.038594,0.339496,0.024308,...,0.339496,0.038378,0.039680,0.028171,0.157049,0.339496,0.150500,0.129681,0.055184,0.020572
1996,0.243917,0.243917,0.164147,0.149064,0.243917,0.123732,0.125073,0.049061,0.059035,0.156946,...,0.243917,0.030823,0.065396,0.163382,0.158244,0.243917,0.243917,0.205795,0.084434,0.010768
1997,0.273236,0.096735,0.133426,0.231087,0.182066,0.059815,0.061674,0.067375,0.148504,0.181868,...,0.024953,0.092274,0.192176,0.273236,0.171343,0.273236,0.092461,0.074493,0.068275,0.029567


In [ ]:
from sklearn.utils import shuffle
combined_data = pd.concat([X_train, y_train], axis=1)
shuffled_data = shuffle(combined_data)
shuffled_data.reset_index(drop=True, inplace=True)
X_train=shuffled_data.iloc[:, :-1]
y_train=shuffled_data.iloc[:, -1]

In [ ]:

class MLFFNN_BN(nn.Module):
  def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
    super(MLFFNN_BN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size1)
    self.fc2 = nn.Linear(hidden_size1, hidden_size2)
    self.fc3 = nn.Linear(hidden_size2, output_size)
    self.activation = nn.Tanh()
    self.bn1 = nn.BatchNorm1d(hidden_size1)
    self.bn2 = nn.BatchNorm1d(hidden_size2)
  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.bn1(x)  # Add BN after activation
    x = self.activation(self.fc2(x))
    x = self.bn2(x)  # Add BN after activation
    x = F.log_softmax(x, dim=1)  # Replace with linear for regression
    return x


In [ ]:

class MLFFNN(nn.Module):
  def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
    super(MLFFNN, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size1)
    self.fc2 = nn.Linear(hidden_size1, hidden_size2)
    self.fc3 = nn.Linear(hidden_size2, output_size)
    self.activation = nn.Tanh()

  def forward(self, x):
    x = self.activation(self.fc1(x))
    x = self.activation(self.fc2(x))
    x = F.log_softmax(x, dim=1)  # Replace with linear for regression
    return x


In [ ]:
def initialize_weights(model):
    """
    Initializes the weights of a model using Xavier initialization.

    Args:
        model (torch.nn.Module): The model to initialize weights for.
    """
    for m in model.modules():
        if isinstance(m, nn.Linear):
            nn.init.xavier_uniform_(m.weight)

In [ ]:

# Example usage
model = MLFFNN(36, 6,20, 5)  # Define network with input size 10, 2 hidden layers of 20 neurons each, and output size 5
modelbn = MLFFNN_BN(36, 6,20, 5)  # Define network with input size 10, 2 hidden layers of 20 neurons each, and output size 5
initialize_weights(model)
initialize_weights(modelbn)


In [ ]:
X_train_tensor = torch.from_numpy(X_train.values).float()
y_train_tensor = torch.from_numpy(y_train.values).long()

In [ ]:

# Hyperparameters
learning_rate = 0.08
batch_size = 500
num_epochs = 10000
early_stopping_threshold = 1e-7

# ... (your data loading logic for train_dataset)

# train_loader = torch.utils.data.DataLoader(combined_data, batch_size=batch_size, shuffle=True)
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor),
                                             batch_size=batch_size, shuffle=True)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:

prev_avg_loss = float('inf')  # Initializing for early stopping
for epoch in range(num_epochs):
  total_loss = float(0)
  for data, target in train_loader:
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output, target)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  avg_loss = total_loss / len(train_loader)
  print(avg_loss)

  if epoch > 0 and abs(prev_avg_loss - avg_loss) < early_stopping_threshold:
    print("Stopping training due to minimal average error change")
    break

  prev_avg_loss = avg_loss  # Store current avg_loss for next iteration comparison

# ... (your validation/testing code)


1.9502054452896118
1.9502054452896118
Stopping training due to minimal average error change


In [ ]:

prev_avg_loss = float('inf')  # Initializing for early stopping
for epoch in range(num_epochs):
  total_loss = float(0)
  for data, target in train_loader:
    optimizer.zero_grad()
    output = modelbn(data)
    loss = criterion(output, target)
    total_loss += loss.item()
    loss.backward()
    optimizer.step()

  avg_loss = total_loss / len(train_loader)
  print(avg_loss)

  if epoch > 0 and abs(prev_avg_loss - avg_loss) < early_stopping_threshold:
    print("Stopping training due to minimal average error change")
    break

  prev_avg_loss = avg_loss  # Store current avg_loss for next iteration comparison

# ... (your validation/testing code)


Streaming output truncated to the last 5000 lines.
2.1915822625160217
2.1924967169761658
2.1924514174461365
2.192083716392517
2.193286657333374
2.196073353290558
2.1924330592155457
2.19295471906662
2.1922457814216614
2.1927821040153503
2.1918697357177734
2.1918433904647827
2.1920111179351807
2.191595137119293
2.194551408290863
2.191653609275818
2.193941056728363
2.1929819583892822
2.191715955734253
2.1927717328071594
2.192977547645569
2.192103385925293
2.194568455219269
2.1961681246757507
2.192556917667389
2.1938175559043884
2.1939547061920166
2.192194104194641
2.1937344670295715
2.1930224299430847
2.1925496459007263
2.19295072555542
2.191947638988495
2.1908902525901794
2.1938623785972595
2.1931490302085876
2.1944552063941956
2.1940693259239197
2.1932244896888733
2.193374216556549
2.193181037902832
2.1927343010902405
2.1931601762771606
2.1936997175216675
2.1941664814949036
2.192183494567871
2.19275438785553
2.193479835987091
2.1952027678489685
2.193365156650543
2.191914200782776
2.1931

In [ ]:
# Assuming X_test and y_test are pandas DataFrames (similar to X_train)

# Convert test data and labels to tensors (if necessary)
X_test_tensor = torch.from_numpy(X_test.values).float()
y_test_tensor = torch.from_numpy(y_test.values).long()

test_loss = 0.0
correct = 0
total = 0
test_dataset =torch.utils.data.TensorDataset(X_test_tensor, y_test_tensor)  # Assuming a custom Dataset class for test data
test_loader = torch.utils.data.DataLoader(test_dataset, shuffle=False)  # Set shuffle=False for testing

with torch.no_grad():
    for data, target in test_loader:  # Assuming you have a test_loader for efficiency
        output = model(data)
        # ... (calculate loss)
        test_loss += loss.item()
        _, predicted = torch.max(output.data, 1)  # Get the index of the maximum probability
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Calculate average test loss and accuracy
test_loss /= total
accuracy = 100. * correct / total

print('Test Loss: {:.6f}'.format(test_loss))
print('Test Accuracy: {:.2f}%'.format(accuracy))


Test Loss: 2.225230
Test Accuracy: 45.29%


In [ ]:

# Assuming X_train and y_train are pandas DataFrames (similar to X_train)

# Convert train data and labels to tensors (if necessary)
X_train_tensor = torch.from_numpy(X_train.values).float()
y_train_tensor = torch.from_numpy(y_train.values).long()

train_loss = 0.0
correct = 0
total = 0
train_dataset =torch.utils.data.TensorDataset(X_train_tensor, y_train_tensor)  # Assuming a custom Dataset class for train data
train_loader = torch.utils.data.DataLoader(train_dataset, shuffle=False)  # Set shuffle=False for training

with torch.no_grad():
    for data, target in train_loader:  # Assuming you have a train_loader for efficiency
        output = model(data)
        # ... (calculate loss)
        train_loss += loss.item()
        _, predicted = torch.max(output.data, 1)  # Get the index of the maximum probability
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Calculate average train loss and accuracy
train_loss /= total
accuracy = 100. * correct / total

print('train Loss: {:.6f}'.format(train_loss))
print('train Accuracy: {:.2f}%'.format(accuracy))



train Loss: 2.225230
train Accuracy: 50.98%


In [ ]:
# Assuming X_valid and y_valid are pandas DataFrames (similar to X_valid)

# Convert valid data and labels to tensors (if necessary)
X_valid_tensor = torch.from_numpy(X_valid.values).float()
y_valid_tensor = torch.from_numpy(y_valid.values).long()

valid_loss = 0.0
correct = 0
total = 0
valid_dataset =torch.utils.data.TensorDataset(X_valid_tensor, y_valid_tensor)  # Assuming a custom Dataset class for valid data
valid_loader = torch.utils.data.DataLoader(valid_dataset, shuffle=False)  # Set shuffle=False for validing

with torch.no_grad():
    for data, target in valid_loader:  # Assuming you have a valid_loader for efficiency
        output = model(data)
        # ... (calculate loss)
        valid_loss += loss.item()
        _, predicted = torch.max(output.data, 1)  # Get the index of the maximum probability
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Calculate average valid loss and accuracy
valid_loss /= total
accuracy = 100. * correct / total

print('valid Loss: {:.6f}'.format(valid_loss))
print('valid Accuracy: {:.2f}%'.format(accuracy))



valid Loss: 2.225230
valid Accuracy: 49.30%
